# Ngonye Falls Flow Analysis


In [110]:
import numpy as np
import pandas as pd

## Setup the Daily data

Load the daily flow series

In [111]:
daily = pd.read_csv("daily.csv")
daily

,Date,Flow
0,01/10/1924,111
1,02/10/1924,111
2,03/10/1924,111
3,04/10/1924,111
4,05/10/1924,111
...,...,...
33963,26/09/2017,201
33964,27/09/2017,201
33965,28/09/2017,201
33966,29/09/2017,201


Index by date and add some other columns for later use. 

Add a column for *WaterYear* which starts on 1st October and runs to 31st September the following year.

In [112]:
daily['Date']=pd.to_datetime(daily['Date'],format="%d/%m/%Y")
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))
daily=daily.drop(['Date'],axis=1)

daily['Year']=daily.index.year
daily['Month']=daily.index.month
daily['Day']=daily.index.day
daily['MonthId']=daily['Year']+daily['Month']/100
daily['WaterYear']=daily.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)

daily

,Flow,Year,Month,Day,MonthId,WaterYear,WaterMonth
Date,,,,,,,
1924-10-01,111,1924,10,1,1924.10,1924.0,1.0
1924-10-02,111,1924,10,2,1924.10,1924.0,1.0
1924-10-03,111,1924,10,3,1924.10,1924.0,1.0
1924-10-04,111,1924,10,4,1924.10,1924.0,1.0
1924-10-05,111,1924,10,5,1924.10,1924.0,1.0
...,...,...,...,...,...,...,...
2017-09-26,201,2017,9,26,2017.09,2016.0,12.0
2017-09-27,201,2017,9,27,2017.09,2016.0,12.0
2017-09-28,201,2017,9,28,2017.09,2016.0,12.0


## Setup the Monthly Data

Load the monthly data.

In [113]:

monthly=daily.groupby(['MonthId','Year','Month']).size().to_frame(name="Days").reset_index(['Month','Year'])
monthly

,Year,Month,Days
MonthId,,,
1924.10,1924,10,31
1924.11,1924,11,30
1924.12,1924,12,31
1925.01,1925,1,31
1925.02,1925,2,28
...,...,...,...
2017.05,2017,5,31
2017.06,2017,6,30
2017.07,2017,7,31


Set the index and add additional columns for later use.

In [114]:
#monthly['Month']=monthly['MonthNo']
#monthly=monthly.drop('MonthNo',1)
#monthly['MonthId']=monthly['Year']+monthly['Month']/100
#monthly=monthly.set_index(pd.Index(monthly['MonthId']))
#monthly=monthly.drop(['MonthId'],axis=1)

monthly['Day']=1
monthly['DateStart']=pd.to_datetime(monthly[['Year','Month','Day']])
monthly=monthly.drop('Day',1)


#monthly['MonthDecimal']=monthly['Year']+monthly['Month']/12
monthly['WaterYear']=monthly.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
monthly['WaterMonth']=monthly.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)


monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth
MonthId,,,,,,
1924.10,1924,10,31,1924-10-01,1924,1
1924.11,1924,11,30,1924-11-01,1924,2
1924.12,1924,12,31,1924-12-01,1924,3
1925.01,1925,1,31,1925-01-01,1924,4
1925.02,1925,2,28,1925-02-01,1924,5
...,...,...,...,...,...,...
2017.05,2017,5,31,2017-05-01,2016,8
2017.06,2017,6,30,2017-06-01,2016,9
2017.07,2017,7,31,2017-07-01,2016,10


## Monthly flow summaries

Add flow summaries to the monthly data

In [115]:
monthly['Flow_min']=daily[['MonthId','Flow']].groupby('MonthId').min()
monthly['Flow_mean']=daily[['MonthId','Flow']].groupby('MonthId').mean()
monthly['Flow_median']=daily[['MonthId','Flow']].groupby('MonthId').median()
monthly['Flow_max']=daily[['MonthId','Flow']].groupby('MonthId').max()
monthly['Flow_range']=monthly['Flow_max']-monthly['Flow_min']
monthly[['Flow_min','Flow_mean','Flow_median','Flow_max','Flow_range']]

,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_range
MonthId,,,,,
1924.10,111,116.096774,111.0,127,16
1924.11,128,154.066667,145.0,191,63
1924.12,193,275.677419,271.0,440,247
1925.01,418,692.774194,596.0,1183,765
1925.02,1108,1193.750000,1151.0,1437,329
...,...,...,...,...,...
2017.05,1636,2195.354839,2162.0,2773,1137
2017.06,678,1118.833333,1121.0,1612,934
2017.07,382,482.709677,466.0,649,267


## Annual Flow

Create a data table for annual (water year) summaries and populate.

In [116]:
yearly=monthly[['WaterYear']].groupby('WaterYear').count()
yearly['YearDecimal']=yearly.index+1.25
yearly['Flow_min']=daily[['WaterYear','Flow']].groupby('WaterYear').min()
yearly['Flow_median']=daily[['WaterYear','Flow']].groupby('WaterYear').median()
yearly['Flow_mean']=daily[['WaterYear','Flow']].groupby('WaterYear').mean()
yearly['Flow_max']=daily[['WaterYear','Flow']].groupby('WaterYear').max()
yearly['Flow_range']=yearly['Flow_max']-yearly['Flow_min']
yearly['Flow_sum']=daily[['WaterYear','Flow']].groupby('WaterYear').sum()

yearly

,YearDecimal,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Flow_sum
WaterYear,,,,,,,
1924,1925.25,111,563.0,992.093151,3452,3341,362114
1925,1926.25,159,495.0,1111.873973,4501,4342,405834
1926,1927.25,193,563.0,990.427397,3313,3120,361506
1927,1928.25,180,512.0,848.065574,2261,2081,310392
1928,1929.25,195,389.0,564.495890,1589,1394,206041
...,...,...,...,...,...,...,...
2012,2013.25,222,690.0,1336.619178,3615,3393,487866
2013,2014.25,195,809.0,1340.526027,3575,3380,489292
2014,2015.25,209,542.0,697.191781,1460,1251,254475


In [117]:
Flow_mean_mean=yearly['Flow_mean'].describe()['mean']
yearly['Flow_mean_pct_var']=(yearly['Flow_mean']-Flow_mean_mean)/Flow_mean_mean*100
Flow_mean_mean

1096.6272871496565

In [118]:
yearly['Flow_mean_5yr_mvCoefVar']=yearly['Flow_mean'].rolling(5,center=True).std()/Flow_mean_mean*100
yearly

,YearDecimal,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Flow_sum,Flow_mean_pct_var,Flow_mean_5yr_mvCoefVar
WaterYear,,,,,,,,,
1924,1925.25,111,563.0,992.093151,3452,3341,362114,-9.532330,NaN
1925,1926.25,159,495.0,1111.873973,4501,4342,405834,1.390325,NaN
1926,1927.25,193,563.0,990.427397,3313,3120,361506,-9.684228,19.171288
1927,1928.25,180,512.0,848.065574,2261,2081,310392,-22.666016,19.695867
1928,1929.25,195,389.0,564.495890,1589,1394,206041,-48.524362,15.786412
...,...,...,...,...,...,...,...,...,...
2012,2013.25,222,690.0,1336.619178,3615,3393,487866,21.884545,32.540110
2013,2014.25,195,809.0,1340.526027,3575,3380,489292,22.240805,24.713616
2014,2015.25,209,542.0,697.191781,1460,1251,254475,-36.423998,24.287307


## Calendar months

Produce summaries of flow by calendar month

In [120]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly['Flow_min']=daily[['WaterMonth','Flow']].groupby('WaterMonth').min()
calmonthly['Flow_mean']=daily[['WaterMonth','Flow']].groupby('WaterMonth').mean()
calmonthly['Flow_median']=daily[['WaterMonth','Flow']].groupby('WaterMonth').median()
calmonthly['Flow_max']=daily[['WaterMonth','Flow']].groupby('WaterMonth').max()
calmonthly['Flow_std']=daily[['WaterMonth','Flow']].groupby('WaterMonth').std()
calmonthly['Flow_coefvar']=(calmonthly['Flow_std']/calmonthly['Flow_mean']*100).round(1)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar
WaterMonth,,,,,,,,
1,Oct,10,105,238.687131,227,470,60.091359,25.2
2,Nov,11,103,275.917921,262,648,80.767132,29.3
3,Dec,12,189,432.308706,409,1128,138.401985,32.0
4,Jan,1,271,719.188692,630,3886,346.203434,48.1
5,Feb,2,389,1347.295013,971,9912,1164.341080,86.4
6,Mar,3,512,2530.635102,2169,9530,1655.915744,65.4
7,Apr,4,685,2965.058065,2960,8540,1354.223410,45.7
8,May,5,421,2219.389178,2155,5673,924.202052,41.6
9,Jun,6,271,1202.737993,1128,3920,551.376925,45.8


## Calendar Month Flow exceedance

Flow exceedance values by calendar month.

P90 is flow which is exceeded for 90% of the time.

In [121]:

calmonthly['Flow_P95']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.05)
calmonthly['Flow_P90']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.1)
#calmonthly['Flow_P80']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.2)
calmonthly['Flow_P75']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.25)
calmonthly['Flow_P50']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.5)
calmonthly['Flow_P25']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.75)
#calmonthly['Flow_P20']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.8)
calmonthly['Flow_P10']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.9)
calmonthly['Flow_P05']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.95)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar,Flow_P95,Flow_P90,Flow_P75,Flow_P50,Flow_P25,Flow_P10,Flow_P05
WaterMonth,,,,,,,,,,,,,,,
1,Oct,10,105,238.687131,227,470,60.091359,25.2,165.890323,174.103226,198.709677,228.612903,274.419355,308.458065,342.858065
2,Nov,11,103,275.917921,262,648,80.767132,29.3,177.353333,200.366667,220.800000,265.466667,310.233333,375.866667,410.033333
3,Dec,12,189,432.308706,409,1128,138.401985,32.0,289.735484,307.870968,346.709677,402.322581,504.354839,563.709677,686.851613
4,Jan,1,271,719.188692,630,3886,346.203434,48.1,435.961290,478.683871,544.064516,620.612903,811.935484,998.477419,1316.012903
5,Feb,2,389,1347.295013,971,9912,1164.341080,86.4,627.457143,691.401724,809.275862,987.500000,1419.607143,2480.138916,3201.028571
6,Mar,3,512,2530.635102,2169,9530,1655.915744,65.4,850.516129,912.812903,1185.516129,2339.709677,3122.677419,4347.703226,5638.677419
7,Apr,4,685,2965.058065,2960,8540,1354.223410,45.7,1084.020000,1258.233333,1925.300000,2959.633333,3698.066667,4655.120000,5104.733333
8,May,5,421,2219.389178,2155,5673,924.202052,41.6,877.354839,1129.574194,1685.032258,2236.290323,2674.322581,3218.361290,3622.948387
9,Jun,6,271,1202.737993,1128,3920,551.376925,45.8,450.420000,631.886667,855.500000,1178.366667,1467.266667,1814.180000,1973.573333


In [125]:
daily.to_csv('prepared_daily.csv')
daily

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar,Flow_P95,Flow_P90,Flow_P75,Flow_P50,Flow_P25,Flow_P10,Flow_P05
WaterMonth,,,,,,,,,,,,,,,
1,Oct,10,105,238.687131,227,470,60.091359,25.2,165.890323,174.103226,198.709677,228.612903,274.419355,308.458065,342.858065
2,Nov,11,103,275.917921,262,648,80.767132,29.3,177.353333,200.366667,220.800000,265.466667,310.233333,375.866667,410.033333
3,Dec,12,189,432.308706,409,1128,138.401985,32.0,289.735484,307.870968,346.709677,402.322581,504.354839,563.709677,686.851613
4,Jan,1,271,719.188692,630,3886,346.203434,48.1,435.961290,478.683871,544.064516,620.612903,811.935484,998.477419,1316.012903
5,Feb,2,389,1347.295013,971,9912,1164.341080,86.4,627.457143,691.401724,809.275862,987.500000,1419.607143,2480.138916,3201.028571
6,Mar,3,512,2530.635102,2169,9530,1655.915744,65.4,850.516129,912.812903,1185.516129,2339.709677,3122.677419,4347.703226,5638.677419
7,Apr,4,685,2965.058065,2960,8540,1354.223410,45.7,1084.020000,1258.233333,1925.300000,2959.633333,3698.066667,4655.120000,5104.733333
8,May,5,421,2219.389178,2155,5673,924.202052,41.6,877.354839,1129.574194,1685.032258,2236.290323,2674.322581,3218.361290,3622.948387
9,Jun,6,271,1202.737993,1128,3920,551.376925,45.8,450.420000,631.886667,855.500000,1178.366667,1467.266667,1814.180000,1973.573333


In [127]:
monthly.to_csv('prepared_monthly.csv')
monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_range
MonthId,,,,,,,,,,,
1924.10,1924,10,31,1924-10-01,1924,1,111,116.096774,111.0,127,16
1924.11,1924,11,30,1924-11-01,1924,2,128,154.066667,145.0,191,63
1924.12,1924,12,31,1924-12-01,1924,3,193,275.677419,271.0,440,247
1925.01,1925,1,31,1925-01-01,1924,4,418,692.774194,596.0,1183,765
1925.02,1925,2,28,1925-02-01,1924,5,1108,1193.750000,1151.0,1437,329
...,...,...,...,...,...,...,...,...,...,...,...
2017.05,2017,5,31,2017-05-01,2016,8,1636,2195.354839,2162.0,2773,1137
2017.06,2017,6,30,2017-06-01,2016,9,678,1118.833333,1121.0,1612,934
2017.07,2017,7,31,2017-07-01,2016,10,382,482.709677,466.0,649,267


In [126]:
yearly.to_csv('prepared_yearly.csv')
yearly

,YearDecimal,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,Flow_sum,Flow_mean_pct_var,Flow_mean_5yr_mvCoefVar
WaterYear,,,,,,,,,
1924,1925.25,111,563.0,992.093151,3452,3341,362114,-9.532330,NaN
1925,1926.25,159,495.0,1111.873973,4501,4342,405834,1.390325,NaN
1926,1927.25,193,563.0,990.427397,3313,3120,361506,-9.684228,19.171288
1927,1928.25,180,512.0,848.065574,2261,2081,310392,-22.666016,19.695867
1928,1929.25,195,389.0,564.495890,1589,1394,206041,-48.524362,15.786412
...,...,...,...,...,...,...,...,...,...
2012,2013.25,222,690.0,1336.619178,3615,3393,487866,21.884545,32.540110
2013,2014.25,195,809.0,1340.526027,3575,3380,489292,22.240805,24.713616
2014,2015.25,209,542.0,697.191781,1460,1251,254475,-36.423998,24.287307


In [129]:
calmonthly.to_csv('prepared_calmonthly.csv')
calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar,Flow_P95,Flow_P90,Flow_P75,Flow_P50,Flow_P25,Flow_P10,Flow_P05
WaterMonth,,,,,,,,,,,,,,,
1,Oct,10,105,238.687131,227,470,60.091359,25.2,165.890323,174.103226,198.709677,228.612903,274.419355,308.458065,342.858065
2,Nov,11,103,275.917921,262,648,80.767132,29.3,177.353333,200.366667,220.800000,265.466667,310.233333,375.866667,410.033333
3,Dec,12,189,432.308706,409,1128,138.401985,32.0,289.735484,307.870968,346.709677,402.322581,504.354839,563.709677,686.851613
4,Jan,1,271,719.188692,630,3886,346.203434,48.1,435.961290,478.683871,544.064516,620.612903,811.935484,998.477419,1316.012903
5,Feb,2,389,1347.295013,971,9912,1164.341080,86.4,627.457143,691.401724,809.275862,987.500000,1419.607143,2480.138916,3201.028571
6,Mar,3,512,2530.635102,2169,9530,1655.915744,65.4,850.516129,912.812903,1185.516129,2339.709677,3122.677419,4347.703226,5638.677419
7,Apr,4,685,2965.058065,2960,8540,1354.223410,45.7,1084.020000,1258.233333,1925.300000,2959.633333,3698.066667,4655.120000,5104.733333
8,May,5,421,2219.389178,2155,5673,924.202052,41.6,877.354839,1129.574194,1685.032258,2236.290323,2674.322581,3218.361290,3622.948387
9,Jun,6,271,1202.737993,1128,3920,551.376925,45.8,450.420000,631.886667,855.500000,1178.366667,1467.266667,1814.180000,1973.573333
